In [1]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd

reading dataframes

In [2]:
spark=SparkSession.builder.master("local[*]").appName('test2').getOrCreate()

In [3]:
dfs=spark.read.option("header", "true").parquet("fhvhv_tripdata_2021-01.parquet")


In [4]:
dfs.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', originating_base_num='B02682', request_datetime=datetime.datetime(2021, 1, 1, 0, 28, 9), on_scene_datetime=datetime.datetime(2021, 1, 1, 0, 31, 42), pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, trip_miles=5.26, trip_time=923, base_passenger_fare=22.28, tolls=0.0, bcf=0.67, sales_tax=1.98, congestion_surcharge=2.75, airport_fee=None, tips=0.0, driver_pay=14.99, shared_request_flag='N', shared_match_flag='N', access_a_ride_flag=' ', wav_request_flag='N', wav_match_flag='N'),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', originating_base_num='B02682', request_datetime=datetime.datetime(2021, 1, 1, 0, 45, 56), on_scene_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DO

defining schemas

In [5]:
dfhv=pd.read_parquet("fhvhv_tripdata_2021-01.parquet")

In [6]:
dfhv=dfhv[['hvfhs_license_num', 'dispatching_base_num', 'pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID', 'shared_request_flag']]
dfhv.rename(columns={'shared_request_flag':'SR_flag'}, inplace=True)

In [7]:
#dfhv.to_csv("fhvhv_tripdata_2021-01.csv")

In [8]:
dfhv.head(100).to_csv('head.csv',index=False)

In [9]:
df_pandas=pd.read_csv('head.csv')

In [10]:
df_pandas.dtypes

hvfhs_license_num       object
dispatching_base_num    object
pickup_datetime         object
dropoff_datetime        object
PULocationID             int64
DOLocationID             int64
SR_flag                 object
dtype: object

In [11]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_flag', StringType(), True)])

In [12]:
from pyspark.sql import types

In [13]:
schema=types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True), 
    types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('pickup_datetime', types.TimestampType(), True), 
    types.StructField('dropoff_datetime', types.TimestampType(), True), 
    types.StructField('PULocationID', types.IntegerType(), True), 
    types.StructField('DOLocationID', types.IntegerType(), True), 
    types.StructField('SR_flag', types.StringType(), True), 
])

In [19]:
df=spark.read.option("header", "true").schema(schema).csv("fhvhv_tripdata_2021-01.csv")
#df=spark.read.option("header", "true").schema(schema).csv("head.csv")


In [20]:
df.head(5)

[Row(hvfhs_license_num='0', dispatching_base_num='HV0003', pickup_datetime=None, dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), PULocationID=None, DOLocationID=230, SR_flag='166'),
 Row(hvfhs_license_num='1', dispatching_base_num='HV0003', pickup_datetime=None, dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), PULocationID=None, DOLocationID=152, SR_flag='167'),
 Row(hvfhs_license_num='2', dispatching_base_num='HV0003', pickup_datetime=None, dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 23, 56), PULocationID=None, DOLocationID=233, SR_flag='142'),
 Row(hvfhs_license_num='3', dispatching_base_num='HV0003', pickup_datetime=None, dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 42, 51), PULocationID=None, DOLocationID=142, SR_flag='143'),
 Row(hvfhs_license_num='4', dispatching_base_num='HV0003', pickup_datetime=None, dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 48, 14), PULocationID=None, DOLocationID=143, SR_flag='78')]

partitioning files

In [21]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_flag: string (nullable = true)



In [22]:
df=df.repartition(24)

In [23]:
df.write.parquet("fhv/2021/01/", mode='overwrite')

dataframe operations with spark

In [24]:
df2=spark.read.parquet("fhv/2021/01/")

In [25]:
df2.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_flag: string (nullable = true)



In [34]:
df=pd.read_parquet("fhvhv_tripdata_2021-01.parquet")

In [38]:
df['pickup_datetime'].value_counts()

pickup_datetime
2021-01-01 00:34:16    23
2021-01-01 02:39:14    23
2021-01-01 00:45:55    22
2021-01-01 02:36:02    22
2021-01-02 16:27:59    22
                       ..
2021-01-18 11:48:07     1
2021-01-18 11:47:47     1
2021-01-18 11:12:22     1
2021-01-18 11:57:30     1
2021-01-31 23:43:54     1
Name: count, Length: 2491317, dtype: int64